In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

### Device configuration

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

### Hyper parameters

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 128
learning_rate = 0.001

### MNIST dataset

In [4]:
train_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    transform=transforms.ToTensor()
)

### Data loader

In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

### Convolutional neural network (two convolutional layers)

In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = ConvNet(num_classes).to(device)

In [8]:
model

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

### Loss and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Train the model

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/469], Loss: 0.1417
Epoch [1/5], Step [200/469], Loss: 0.0851
Epoch [1/5], Step [300/469], Loss: 0.0385
Epoch [1/5], Step [400/469], Loss: 0.0875
Epoch [2/5], Step [100/469], Loss: 0.0605
Epoch [2/5], Step [200/469], Loss: 0.0290
Epoch [2/5], Step [300/469], Loss: 0.0510
Epoch [2/5], Step [400/469], Loss: 0.0050
Epoch [3/5], Step [100/469], Loss: 0.0322
Epoch [3/5], Step [200/469], Loss: 0.0228
Epoch [3/5], Step [300/469], Loss: 0.0094
Epoch [3/5], Step [400/469], Loss: 0.0575
Epoch [4/5], Step [100/469], Loss: 0.0297
Epoch [4/5], Step [200/469], Loss: 0.0028
Epoch [4/5], Step [300/469], Loss: 0.0128
Epoch [4/5], Step [400/469], Loss: 0.0356
Epoch [5/5], Step [100/469], Loss: 0.0105
Epoch [5/5], Step [200/469], Loss: 0.0042
Epoch [5/5], Step [300/469], Loss: 0.0206
Epoch [5/5], Step [400/469], Loss: 0.0271


### Test the model

In [14]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.47 %
